In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import numpy as np
import pandas as pd
import hvplot.pandas
import plotly.express as px
import tensorflow as tf

In [23]:
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from imblearn.metrics import classification_report_imbalanced

In [24]:
car_clean_df = pd.read_csv("../Resources/car_clean.csv")
car_clean_df.head()

,Unnamed: 0,manufacturer,model,year,selling_price,fuel,seller_type,mileage
0,0,Maruti,Swift Dzire VDI,2014,450000,Diesel,Individual,23.40
1,1,Skoda,Rapid 1.5 TDI Ambition,2014,370000,Diesel,Individual,21.14
2,2,Honda,City 2017-2020 EXi,2006,158000,Petrol,Individual,17.70
3,3,Hyundai,i20 Sportz Diesel,2010,225000,Diesel,Individual,23.00
4,4,Maruti,Swift VXI BSIII,2007,130000,Petrol,Individual,16.10


In [25]:
# Generate our categorical variable lists
df_cat = car_clean_df.dtypes[car_clean_df.dtypes == "object"].index.tolist()

car_clean_df[df_cat].nunique()

manufacturer      31
model           1983
fuel               3
seller_type        3
dtype: int64

In [26]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(car_clean_df[df_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(df_cat)
encode_df.head()

,manufacturer_Ambassador,manufacturer_Ashok,manufacturer_Audi,manufacturer_BMW,manufacturer_Chevrolet,manufacturer_Daewoo,manufacturer_Datsun,manufacturer_Fiat,manufacturer_Force,manufacturer_Ford,...,model_i20 Sportz AT 1.4,model_i20 Sportz Diesel,model_i20 Sportz Option 1.2,model_i20 Sportz Option 1.4 CRDi,fuel_Diesel,fuel_Other,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [27]:
# Merge one-hot encoded features and drop the originals
car_clean_df = car_clean_df.merge(encode_df,left_index=True, right_index=True)
car_clean_df = car_clean_df.drop(df_cat, 1)
car_clean_df.head()

,Unnamed: 0,year,selling_price,mileage,manufacturer_Ambassador,manufacturer_Ashok,manufacturer_Audi,manufacturer_BMW,manufacturer_Chevrolet,manufacturer_Daewoo,...,model_i20 Sportz AT 1.4,model_i20 Sportz Diesel,model_i20 Sportz Option 1.2,model_i20 Sportz Option 1.4 CRDi,fuel_Diesel,fuel_Other,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer
0,0,2014,450000,23.40,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,2014,370000,21.14,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2,2006,158000,17.70,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,3,2010,225000,23.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,4,2007,130000,16.10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


### Predicting Diesel

In [28]:
# Split our preprocessed data into our features and target arrays
y = car_clean_df["fuel_Diesel"].values
X = car_clean_df.drop(["fuel_Diesel"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 135
hidden_nodes_layer2 = 68

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 135)               273240    
                                                                 
 dense_4 (Dense)             (None, 68)                9248      
                                                                 
 dense_5 (Dense)             (None, 1)                 69        
                                                                 
Total params: 282,557
Trainable params: 282,557
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', 
    period=5)

In [32]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
186/186 [==============================] - 2s 5ms/step - loss: 1.9348 - accuracy: 0.8069
Epoch 2/100
186/186 [==============================] - 1s 4ms/step - loss: 1.0269 - accuracy: 0.9196
Epoch 3/100
186/186 [==============================] - 1s 4ms/step - loss: 0.6853 - accuracy: 0.9428
Epoch 4/100
186/186 [==============================] - 1s 4ms/step - loss: 0.5486 - accuracy: 0.9577
Epoch 5/100
180/186 [============================>.] - ETA: 0s - loss: 0.4732 - accuracy: 0.9653
Epoch 5: saving model to checkpoints\weights.05.hdf5
186/186 [==============================] - 1s 5ms/step - loss: 0.4624 - accuracy: 0.9659
Epoch 6/100
186/186 [==============================] - 1s 4ms/step - loss: 0.3666 - accuracy: 0.9712
Epoch 7/100
186/186 [==============================] - 1s 4ms/step - loss: 0.3062 - accuracy: 0.9772
Epoch 8/100
186/186 [==============================] - 1s 5ms/step - loss: 0.5412 - accuracy: 0.9524
Epoch 9/100
186/186 [==============================] -

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

62/62 - 1s - loss: 0.3854 - accuracy: 0.9616 - 583ms/epoch - 9ms/step
Loss: 0.3854294419288635, Accuracy: 0.961557924747467


### Predicting Other Fuel

In [34]:
# Split our preprocessed data into our features and target arrays
y = car_clean_df["fuel_Other"].values
X = car_clean_df.drop(["fuel_Other"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [35]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 135
hidden_nodes_layer2 = 68

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 135)               273240    
                                                                 
 dense_7 (Dense)             (None, 68)                9248      
                                                                 
 dense_8 (Dense)             (None, 1)                 69        
                                                                 
Total params: 282,557
Trainable params: 282,557
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', 
    period=5)

In [38]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
186/186 [==============================] - 2s 5ms/step - loss: 0.0958 - accuracy: 0.9671
Epoch 2/100
186/186 [==============================] - 1s 5ms/step - loss: 0.0083 - accuracy: 0.9983
Epoch 3/100
186/186 [==============================] - 1s 5ms/step - loss: 0.0012 - accuracy: 0.9998
Epoch 4/100
186/186 [==============================] - 1s 5ms/step - loss: 4.1280e-04 - accuracy: 1.0000
Epoch 5/100
180/186 [============================>.] - ETA: 0s - loss: 2.1531e-04 - accuracy: 1.0000
Epoch 5: saving model to checkpoints\weights.05.hdf5
186/186 [==============================] - 1s 6ms/step - loss: 2.1371e-04 - accuracy: 1.0000
Epoch 6/100
186/186 [==============================] - 1s 5ms/step - loss: 1.2780e-04 - accuracy: 1.0000
Epoch 7/100
186/186 [==============================] - 1s 5ms/step - loss: 8.5643e-05 - accuracy: 1.0000
Epoch 8/100
186/186 [==============================] - 1s 6ms/step - loss: 6.0909e-05 - accuracy: 1.0000
Epoch 9/100
186/186 [=========

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

62/62 - 0s - loss: 0.0485 - accuracy: 0.9965 - 430ms/epoch - 7ms/step
Loss: 0.04853169620037079, Accuracy: 0.9964593052864075


### Predicting Petrol Fuel

In [40]:
# Split our preprocessed data into our features and target arrays
y = car_clean_df["fuel_Petrol"].values
X = car_clean_df.drop(["fuel_Petrol"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [41]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 135
hidden_nodes_layer2 = 68

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 135)               273240    
                                                                 
 dense_10 (Dense)            (None, 68)                9248      
                                                                 
 dense_11 (Dense)            (None, 1)                 69        
                                                                 
Total params: 282,557
Trainable params: 282,557
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', 
    period=5)

In [44]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
186/186 [==============================] - 2s 5ms/step - loss: 0.2745 - accuracy: 0.8771
Epoch 2/100
186/186 [==============================] - 1s 5ms/step - loss: 0.0061 - accuracy: 0.9988
Epoch 3/100
186/186 [==============================] - 1s 5ms/step - loss: 7.2320e-04 - accuracy: 1.0000
Epoch 4/100
186/186 [==============================] - 1s 5ms/step - loss: 3.2155e-04 - accuracy: 1.0000
Epoch 5/100
181/186 [============================>.] - ETA: 0s - loss: 1.9700e-04 - accuracy: 1.0000
Epoch 5: saving model to checkpoints\weights.05.hdf5
186/186 [==============================] - 1s 5ms/step - loss: 1.9625e-04 - accuracy: 1.0000
Epoch 6/100
186/186 [==============================] - 1s 5ms/step - loss: 1.3213e-04 - accuracy: 1.0000
Epoch 7/100
186/186 [==============================] - 1s 5ms/step - loss: 9.4742e-05 - accuracy: 1.0000
Epoch 8/100
186/186 [==============================] - 1s 5ms/step - loss: 7.0949e-05 - accuracy: 1.0000
Epoch 9/100
186/186 [=====

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

62/62 - 0s - loss: 0.0092 - accuracy: 0.9960 - 291ms/epoch - 5ms/step
Loss: 0.009150318801403046, Accuracy: 0.9959534406661987
